# To do:

 - Both
     - Fix Injuries data
         - Find a better source for roster data (I found, work on the plyr_pos_xref notebook)
     - Signal Opp Injuries
 - Mins
 - PTS
     - Find more effective way to signal Defensive stats
 - Res_PTS
     - Try some sort of defensive rk L5 - d_rk today feature to measure difficulty increase/decrease
     - Create some short term Lx Average features for the target columns (Res_PTS/Bet)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import duckdb
import warnings
import os
import json

import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
from scipy.stats import randint, uniform

from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.inspection import PartialDependenceDisplay

import joblib
import warnings
from datetime import datetime, timedelta

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

categories = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']
con = duckdb.connect(database=":memory:")

cwd = os.path.abspath(os.getcwd()).replace("\\", "/")
if cwd.startswith("C:/Users/Rodolfo/"):
    RUN_LOCATION = "local"
    MDL_PATH = "../ML_models/dev"
else:
    RUN_LOCATION = "cloud"
    MDL_PATH = "../ML_models"
time_offset = {"local": 3, "cloud": -5}
now = str((datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]) + timedelta(hours=-3)).date())
print(f"Today's date:", now)

tgt_stat = "PTS"
print('Target Stat:', tgt_stat)

Today's date: 2026-01-29
Target Stat: PTS


In [2]:
%run ./common_utils.ipynb

# ML Functions

In [3]:
def feature_importance(model, all_features):
    booster = model.get_booster()
    score = booster.get_score(importance_type="gain")

    df_importance = pd.DataFrame({
            "feature": all_features,
            "importance": [score.get(f, 0.0) for f in all_features]
        }).sort_values("importance", ascending=False).reset_index(drop=True)
    
    df_importance['pct'] = df_importance.importance.cumsum() / df_importance.importance.sum()
    df_importance['importance'] = df_importance['importance'].map('{:.4f}'.format)
    if df_importance.shape[0] >= 50:
        with pd.option_context('display.max_rows', None):
            display(df_importance)
    else:
        display(df_importance)
    
    xgb.plot_importance(model)
    plt.show()

In [4]:
def compute_sample_weights(df, decay=0.99):
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    max_date = df['Date'].max()
    df["days_old"] = (max_date - df['Date']).dt.days
    weights = decay ** df["days_old"]
    
    return weights.values

In [5]:
def quantile_loss(y_true, y_pred, q):
    diff = y_true - y_pred
    return np.mean(np.maximum(q * diff, (q - 1) * diff))

In [6]:
def create_baseline_model(df, pred_col, DFS):
    
    train_df, val_df, test_df = DFS

    if pred_col == 'MP':
        print('Minutes Model')
        feature_cols = [
            'MP_L3_avg', 'MP_L5_avg', 'MP_L10_avg', 'game_spread_type'
        ]
    else:
        print(f'{pred_col} Stats Model')
        feature_cols = [
            'MP_L5_avg',
            'MP_L10_avg',
            f'{pred_col}_last_3_avg', f'{pred_col}_last_5_avg', f'{pred_col}_last_10_avg',
            f'Def_{pred_col}', f'Def_L5_{pred_col}'
        ]
    
    print('Train:', len(train_df), '/ Validation:', len(val_df), '/ Test:', len(test_df))
    
    X_train, y_train = train_df[feature_cols], train_df[pred_col]
    X_val,   y_val   = val_df[feature_cols],   val_df[pred_col]
    X_test,  y_test  = test_df[feature_cols],  test_df[pred_col]

    # Convert to DMatrix (XGBoost internal format)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval   = xgb.DMatrix(X_val, label=y_val)
    dtest  = xgb.DMatrix(X_test, label=y_test)

    params = {
        "objective": "reg:squarederror",
        "max_depth": 5,
        "learning_rate": 0.05,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "seed": 42
    }

    # Train using native XGBoost API with early stopping
    evals = [(dtrain, "train"), (dval, "val")]
    bst = xgb.train(
        params,
        dtrain,
        num_boost_round=500,
        evals=evals,
        early_stopping_rounds=50,
        verbose_eval=False
    )

    # Predict on test set
    preds = bst.predict(dtest)

    rmse = np.sqrt(mean_squared_error(y_test, preds))
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)

    print("RMSE:", rmse)
    print("MAE:", mae)
    print("R²:", r2)
    
    return bst

In [7]:
def hyperparam_tuning(DFS, pred_col, is_classification=False, quantile=False, n_iter=20, early_stopping_rounds=50, decay=1, q_val=0.5):
    """
    Hyperparameter tuning for XGBRegressor or XGBClassifier using native XGBoost API
    """
    train_df, val_df, test_df = DFS
    feature_cols = [col for col in train_df.columns if col not in ['Season', 'Date', pred_col]]
    X_train, y_train = train_df[feature_cols], train_df[pred_col]
    X_val,   y_val   = val_df[feature_cols],   val_df[pred_col]
    X_test,  y_test  = test_df[feature_cols],  test_df[pred_col]

    # Sample Weights (decay < 1)
    w_train = compute_sample_weights(train_df, decay=decay)
    w_val   = compute_sample_weights(val_df, decay=decay)
    dtrain = xgb.DMatrix(X_train, label=y_train, weight=w_train, enable_categorical=True)
    dval   = xgb.DMatrix(X_val, label=y_val, weight=w_val, enable_categorical=True)
    dtest  = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

    # Hyperparameter search space
    param_dist = {
        "n_estimators": randint(300, 1500),
        "learning_rate": uniform(0.005, 0.08),
        "max_depth": randint(3, 6),
        "min_child_weight": randint(3, 10),
        "subsample": uniform(0.7, 0.3),
        "colsample_bytree": uniform(0.5, 0.5),
        "gamma": uniform(0, 0.8),
        "reg_lambda": uniform(0, 5),
        "reg_alpha": uniform(0, 1)
    }

    # Generate n_iter random parameter sets
    param_list = []
    for _ in range(n_iter):
        sample = {k: (v.rvs() if hasattr(v, "rvs") else v) for k,v in param_dist.items()}
        sample['n_estimators'] = int(sample['n_estimators'])
        sample['max_depth'] = int(sample['max_depth'])
        sample['min_child_weight'] = int(sample['min_child_weight'])
        param_list.append(sample)

    best_score = float('inf') if not is_classification else 0
    best_params = None
    best_bst = None

    for i, params in enumerate(param_list):
        print(f"\nTrial {i+1}/{n_iter}: {params}")
        num_boost_round = params.pop('n_estimators')

        # Set objective based on regression or classification
        if is_classification:
            params.update({
                "objective": "binary:logistic",
                "enable_categorical": True,
                "eval_metric": "logloss",
                "tree_method": "hist",
                "device": "cuda",
                "seed": 42
            })
        elif quantile:
            params.update({
                "objective": "reg:quantileerror",
                "quantile_alpha": q_val, 
                "enable_categorical": True,
                "tree_method": "hist",
                "device": "cuda",
                "seed": 42
            })

        else:
            params.update({
                "objective": "reg:squarederror",
                "enable_categorical": True,
                "tree_method": "hist",
                "device": "cuda",
                "seed": 42
            })

        evals = [(dtrain, 'train'), (dval, 'val')]
        bst = xgb.train(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            evals=evals,
            early_stopping_rounds=early_stopping_rounds,
            verbose_eval=False
        )

        # Validation scoring
        val_preds = bst.predict(dval, iteration_range=(0, bst.best_iteration))
        if is_classification:
            val_class = (val_preds > 0.5).astype(int)
            score = (val_class == y_val.values).mean()  # accuracy
            print(f"Validation Accuracy: {score:.4f}")
            if score > best_score:
                best_score = score
                best_params = params.copy()
                best_bst = bst
        elif quantile:
            q_loss = quantile_loss(y_val.values, val_preds, q_val)
            print(f"Validation Quantile Loss (q_val={q_val}): {q_loss:.4f}")
            if q_loss < best_score:
                best_score = q_loss
                best_params = params.copy()
                best_bst = bst
        else:
            mae = mean_absolute_error(y_val, val_preds)
            print(f"Validation MAE: {mae:.4f}")
            if mae < best_score:
                best_score = mae
                best_params = params.copy()
                best_bst = bst

    print("\nBest score:", best_score)
    print("Best parameters:", best_params)

    # Test predictions
    test_preds = best_bst.predict(dtest, iteration_range=(0, best_bst.best_iteration))
    if is_classification:
        test_class = (test_preds > 0.5).astype(int)
        acc = (test_class == y_test.values).mean()
        print("\nTest Accuracy:", acc)
    else:
        print("\nTest Metrics:")
        if quantile:
            ql = quantile_loss(y_test, test_preds, q_val)
            print("Quantile loss:", ql)
            coverage = np.mean(y_test <= test_preds)
            print(f"Coverage for q_val={q_val}: {coverage:.2f}")
        else:
            print("RMSE:", np.sqrt(mean_squared_error(y_test, test_preds)))
            print("MAE:", mean_absolute_error(y_test, test_preds))
            print("R²:", r2_score(y_test, test_preds))

    return best_params

In [8]:
def refit_model(df, pred_col, params_file, min_train_days=0, rolling_window=None, decay=1):
    df = df.sort_values("Date")
    dates = df["Date"].unique()
    print(f'Rows: {df.shape[0]}, Dates: {len(dates)}, min_train_days: {min_train_days}')

    feature_cols = [c for c in df.columns if c not in ["Season", "Date", pred_col]]

    # Load hyperparameters
    with open(f"{MDL_PATH}/{params_file}.json", "r") as f:
        loaded_params = json.load(f)

    preds, actuals, dates_out, predictions = [], [], [], []
    total_iters = len(dates) - min_train_days

    for idx, i in enumerate(range(min_train_days, len(dates)), start=1):
        test_date = dates[i]
        test_season = df.loc[df["Date"] == test_date, "Season"].iloc[0]

        if rolling_window:
            train_start_idx = max(0, i - rolling_window)
        else:
            train_start_idx = 0

        train_dates = dates[train_start_idx:i]

        train_df = df[df["Date"].isin(train_dates)]
        test_df  = df[df["Date"] == test_date]

        if test_df.empty:
            continue

        X_train, y_train = train_df[feature_cols], train_df[pred_col]
        X_test, y_test   = test_df[feature_cols], test_df[pred_col]

        predictions.append(test_df)
        if pred_col == 'Bet':
            model = XGBClassifier(**loaded_params)
            model.fit(X_train, y_train)
            
            test_df['pred_prob'] = model.predict_proba(X_test)[:,1]
            test_df['pred_class'] = (test_df['pred_prob'] > 0.5).astype(int)
        else:
            model = XGBRegressor(**loaded_params)
            sample_weights = compute_sample_weights(train_df, decay=decay)
            model.fit(X_train, y_train, sample_weight=sample_weights)

            y_pred = model.predict(X_test)
            preds.extend(y_pred)
            actuals.extend(y_test.values)
            dates_out.extend([test_date] * len(y_pred))

        if idx % max(1, total_iters // 20) == 0:
            pct = 100 * idx / total_iters
            print(f"Progress: {pct:6.2f}% ({idx}/{total_iters})")
            
    results = pd.concat(predictions)
    if pred_col == 'Res_PTS':
        results['Actuals'] = actuals
        results['Predictions'] = preds
        mae = mean_absolute_error(actuals, preds)
        print("Walk-forward MAE:", mae)
        results["Correct_Direction"] = (np.sign(results["Predictions"]) == np.sign(results["Actuals"])).astype(int)
        for t in [0, 1, 2, 3]:
            subset = results[results["Predictions"].abs() >= t]
            acc = subset["Correct_Direction"].mean() if len(subset) > 0 else np.nan
            print(f"|Pred| >= {t}: accuracy = {acc:.3f}, n = {len(subset)}")
    elif pred_col == 'Bet':
        cm = confusion_matrix(results['Bet'], results['pred_class'])
        print("Confusion Matrix:\n", cm)
        report = classification_report(results['Bet'], results['pred_class'])
        print("Classification Report:\n", report)
        auc = roc_auc_score(results['Bet'], results['pred_prob'])
        print(f"ROC AUC: {auc:.3f}")

        high_confidence = results.copy()
        high_confidence['pred_prob'] = np.where(high_confidence.pred_prob > 0.5, 1 - high_confidence.pred_prob, high_confidence.pred_prob)
        high_confidence = high_confidence[high_confidence['pred_prob'] <= 0.3]
        if len(high_confidence) > 0:
            hit_rate = (high_confidence['pred_class'] == high_confidence['Bet']).mean()
            print(f"High-confidence hit rate (<= 0.3 & >= 0.7): {hit_rate:.2f}")

    else:
        results['Actuals'] = actuals
        results['Predictions'] = preds
        if loaded_params['objective'] == 'reg:quantileerror':
            ql = quantile_loss(results['Actuals'], results['Predictions'], loaded_params['quantile_alpha'])
            print("Quantile loss:", ql)
            coverage = np.mean(results['Actuals'] <= results['Predictions'])
            print(f"Coverage for q_val={loaded_params['quantile_alpha']}: {coverage:.2f}")
        else:
            mae = mean_absolute_error(actuals, preds)
            rmse = np.sqrt(mean_squared_error(actuals, preds))
            r2 = r2_score(actuals, preds)
            print(f"Walk-forward RMSE: {rmse:.3f}")
            print(f"Walk-forward MAE: {mae:.3f}")
            print(f"Walk-forward R²: {r2:.3f}")

    return model, results

### Create Base df

In [9]:
def load_df(file_name):
    df = pd.DataFrame()
    for i in [2021, 2022, 2023, 2024, 2025]:
        df_temp = pd.read_csv(f"../tables/{i}/{file_name}.csv")
        df_temp['Season'] = i
        df = pd.concat([df, df_temp])
        
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df.Date)
    if file_name == "season_gamelogs":
        df = df[~df[['Date', 'Team', 'Player']].duplicated(keep='last')]
    
    return df

In [10]:
# Load dfs
df = load_df('nba_schedule')
df2 = load_df('season_gamelogs')
# df3 = load_df('REPLACE ME')
df4 = load_df('injuries')
df5 = load_df('plyr_pos_xref')
df6 = load_df('daily_lineups')
gmlog_cols = ['game_id', 'Player', 'MP', 'PF', 'PTS', 'FG', 'FGA', 'FT', 'FTA', '3PM', '3PA', 'ORB', 'TOV']
df7 = load_df('h1_season_gamelogs')[gmlog_cols].rename(columns={"MP": "MP_h1", "PTS": "PTS_h1", "FG": "FG_h1", "FGA": "FGA_h1", "FT": "FT_h1", "FTA": "FTA_h1", "3PM": "TPM_h1", "3PA": "TPA_h1", "PF": "PF_h1", "TOV": "TOV_h1", "ORB": "ORB_h1"})
df8 = load_df('h2_season_gamelogs')[gmlog_cols].rename(columns={"MP": "MP_h2", "PTS": "PTS_h2", "FG": "FG_h2", "FGA": "FGA_h2", "FT": "FT_h2", "FTA": "FTA_h2", "3PM": "TPM_h2", "3PA": "TPA_h2", "PF": "PF_h2", "TOV": "TOV_h2", "ORB": "ORB_h2"})
df9 = load_df('q1_season_gamelogs')[gmlog_cols].rename(columns={"MP": "MP_q1", "PTS": "PTS_q1", "FG": "FG_q1", "FGA": "FGA_q1", "FT": "FT_q1", "FTA": "FTA_q1", "3PM": "TPM_q1", "3PA": "TPA_q1", "PF": "PF_q1", "TOV": "TOV_q1", "ORB": "ORB_q1"})
df10 = load_df('q2_season_gamelogs')[gmlog_cols].rename(columns={"MP": "MP_q2", "PTS": "PTS_q2", "FG": "FG_q2", "FGA": "FGA_q2", "FT": "FT_q2", "FTA": "FTA_q2", "3PM": "TPM_q2", "3PA": "TPA_q2", "PF": "PF_q2", "TOV": "TOV_q2", "ORB": "ORB_q2"})
df11 = load_df('q3_season_gamelogs')[gmlog_cols].rename(columns={"MP": "MP_q3", "PTS": "PTS_q3", "FG": "FG_q3", "FGA": "FGA_q3", "FT": "FT_q3", "FTA": "FTA_q3", "3PM": "TPM_q3", "3PA": "TPA_q3", "PF": "PF_q3", "TOV": "TOV_q3", "ORB": "ORB_q3"})
df12 = load_df('q4_season_gamelogs')[gmlog_cols].rename(columns={"MP": "MP_q4", "PTS": "PTS_q4", "FG": "FG_q4", "FGA": "FGA_q4", "FT": "FT_q4", "FTA": "FTA_q4", "3PM": "TPM_q4", "3PA": "TPA_q4", "PF": "PF_q4", "TOV": "TOV_q4", "ORB": "ORB_q4"})

df_mtch = df[['Season', 'Date', 'AwayABV', 'HomeABV', 'AwayPTS', 'HomePTS', 'AwayB2B', 'HomeB2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
df_mtch['Team_type'] = 'Away'
df_mtch = df_mtch.rename(columns={"AwayABV": "Team", "HomeABV": "Opp", "AwayB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm', 'Team_type']]
df_mtch2 = df_mtch.copy().rename(columns={"Team": "Opp", "Opp": "Team", "HomeB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
df_mtch2['Team_type'] = 'Home'
df = pd.concat([df_mtch, df_mtch2])
df = df.sort_values(["Team", "Date"])
df['team_game_num'] = df.groupby(["Team", "Season"]).cumcount() + 1
df['Spread'] = np.where(df.Team_type == 'Home', df.AwayPTS - df.HomePTS, df.HomePTS - df.AwayPTS)
df['Total'] = df.AwayPTS + df.HomePTS
df['is_Win'] = np.where(df.Spread > 0, 1, 0)
df['Szn_Wins'] = df.groupby(['Season', 'Team'])['is_Win'].cumsum()
df = df.merge(df5, on=['Season', 'Team'])

df2 = df2.rename(columns={"3PM": "TPM", "3PA": "TPA", "3P%": "TP%", "TRB": "REB"})
df2['PR'] = df2.PTS + df2.REB 
df2['PA'] = df2.PTS + df2.AST
df2['RA'] = df2.REB + df2.AST
df2['PRA'] = df2.PTS + df2.REB + df2.AST
df2['STL_BLK'] = df2.STL + df2.BLK
df = df.merge(df2.drop(['Pos', 'Opp', 'Team_type'], axis=1), on=['Season', 'Date', 'Team', 'Player'], how='left')

df = df.merge(df4[['Date', 'Team', 'Player', 'Status']], on=['Date', 'Team', 'Player'], how='left')
df['Status'] = np.where((df.Active == 1) & (df.Status.isnull()), 'Available', df.Status)
df['Status'] = np.where((df.Active == 0), 'Out', df.Status)
df['Status'] = np.where((df.Status == 'Out') & (df.Active != 0), 'Available', df.Status)

df6['role'] = 1
df = df.merge(df6.drop('Pos', axis=1), on=['Season', 'Date', 'Team', 'Player'], how='left')
df['role'] = df.role.fillna(2).astype(int)

# Add gmlog splits
df_gmlog_comb = df7.merge(df8, on=['game_id', 'Player'])
for df_loop in (df9, df10, df11, df12):
    df_gmlog_comb = df_gmlog_comb.merge(df_loop, on=['game_id', 'Player'])
df = df.merge(df_gmlog_comb, on=['game_id', 'Player'], how='left')

df_lines = pd.read_csv(f"../tables/{YEAR}/parlay_lines.csv")
df_lines['Date'] = pd.to_datetime(df_lines.Date)
df_lines = df_lines[~(df_lines.Team.isnull()) & ~(df_lines.PTS_line.isnull())].drop(['Pos', 'Spread', 'Total'], axis=1)
df = df.merge(df_lines, on=['Date', 'Team', 'Player'], how='left')
df['Res_PTS'] = df.PTS - df.PTS_line

df = df.sort_values(['Season', 'Date', 'Team', 'Player']).reset_index(drop=True)
df_td = df[df.Date == now]
df = df[(df.Active == 1) & (df.MP > 0)]
df_pred = df.copy()
df_pred = pd.concat([df_pred, df_td])
print('base df created', datetime.now())

base df created 2026-01-29 20:16:45.544629


### Feature Engineering Helper Functions

In [11]:
def create_df_missing(df, pred_col):

    df3 = load_df('season_gamelogs')
    df3 = df3.rename(columns={"3PM": "TPM", "3PA": "TPA", "3P%": "TP%", "TRB": "REB"}).drop(['Pos', 'Opp'], axis=1)
    df4 = load_df('injuries')
    
    # Fill missing games from injuries.csv
    team_games = df_pred[['Season', 'Team', 'Date']].drop_duplicates()
    players = df_pred[['Season','Player','Team']].drop_duplicates()
    fabricated = (players.sort_values('Season').groupby('Player', as_index=False).last())
    fabricated['Season'] = fabricated['Season'] + 1
    players = pd.concat([players, fabricated], ignore_index=True).drop_duplicates(['Season','Player','Team'])
    expanded = team_games.merge(players, on=['Season', 'Team'], how='left')

    df5 = load_df('plyr_pos_xref')

    expanded = expanded.merge(df3[['Season', 'Player', 'Date', 'MP']], on=['Season', 'Player', 'Date'], how='left').drop_duplicates(['Season', 'Date', 'Player', 'Team'])
    expanded = expanded[(expanded.MP.isnull()) & (expanded.Date != now)].drop('MP', axis=1)
    expanded = pd.concat([expanded, df4[df4.Status == 'Out'][['Season', 'Team', 'Date', 'Player']]])
    df4 = df4.merge(expanded, on=['Season', 'Date', 'Team', 'Player'], how='right')

    # Grab outs from players season gamelogs
    df4 = df4.merge(df3, on=['Season', 'Date', 'Team', 'Player'], how='outer')
    df4['Status'] = np.where(((df4.Active == 1) | (df4.MP > 0)), 'Available', df4.Status)
    df4['Status'] = np.where(((df4.Active == 0) | (df4.MP == 0) | (df4.MP.isnull())), 'Out', df4.Status)
    df4['Status'] = np.where((df4.Status == 'Out') & (df4.MP > 0), 'Available', df4.Status)
    df4['Status'] = np.where((df4.Status != 'Out') & (df4.MP == 0), 'Out', df4.Status)
    df4 = df4[df4.Status == 'Out'][['Season', 'Date', 'Team', 'Player']].drop_duplicates()
    
    df_missing = df[['Season', 'Date', 'Team', 'Player', 'role', pred_col]].copy()
    df_missing[f'{pred_col}_L10'] = (
        df_missing.sort_values(['Player', 'Date']).groupby(['Player','Season'])[pred_col].shift(1)
                  .transform(lambda x: x.rolling(10, min_periods=10).mean())
    )
    df_missing['role_L10_mode'] = (
        df_missing.sort_values(['Player', 'Date'])
            .groupby(['Player', 'Season'])['role'].shift(1)
            .transform(lambda x: x.rolling(10, min_periods=10)
                            .apply(lambda y: np.bincount(y.astype(np.int8), minlength=4).argmax(), raw=True))
    )
    df_missing = pd.merge_asof(df4, df_missing[["Season", "Player", "Date", "role", "role_L10_mode", f"{pred_col}_L10"]], 
                      on="Date", by=["Player", "Season"], direction="backward", allow_exact_matches=True).dropna()   
    df_missing = df_missing.merge(df5, on=['Season', 'Team', 'Player'])
    
    # Filter out old injuries
    df_missing = df_missing.sort_values(["Season", "Team", "Player", "Date"])
    df_missing["team_game_num"] = (df_missing.groupby(["Season", "Team"])["Date"].rank(method="dense").astype(int))
    df_missing["game_break"] = (df_missing.groupby(["Season", "Team", "Player"])["team_game_num"].diff().ne(1))
    df_missing["streak_id"] = (df_missing.groupby(["Season", "Team", "Player"])["game_break"].cumsum())
    df_missing["consecutive_games"] = (df_missing.groupby(["Season", "Team", "Player", "streak_id"]).cumcount().add(1))
    df_missing["eligible_today"] = (df_missing["consecutive_games"] <= 10).astype(int)
    df_missing["role_for_count"] = np.where(df_missing["eligible_today"] == 1, df_missing["role_L10_mode"], np.nan)    
    df_missing[f'{pred_col}_L10'] = np.where(df_missing['role_for_count'] == 1, df_missing[f'{pred_col}_L10'], 0)

#     display(df_missing[(df_missing.Team == 'CLE') & (df_missing.Date == '2026-01-23')].tail(10))

    out_minutes = (
    df_missing
      .groupby(["Season", "Date", "Team"])
      .agg(
          tgt_available=(f"{pred_col}_L10", lambda x: x.sum()),
          starters_out=("role_for_count", lambda x: (x == 1).sum())
      )
      .reset_index()
    ).rename(columns={"tgt_available": f"team_{pred_col}_available"})

    return out_minutes

In [12]:
def filter_out_early_exits(df):
    for N in [3, 5, 10]:
        df[f'MP_L{N}_avg'] = (
            df.sort_values(['Player', 'Date']).groupby(['Player', 'Season'])['MP'].shift(1)
             .rolling(window=N, min_periods=N)
             .mean()
        )
    df['MP_base'] = df[['MP_L3_avg', 'MP_L5_avg', 'MP_L10_avg']].mul([0.15, 0.25, 0.60]).sum(axis=1, skipna=True) / df[['MP_L3_avg', 'MP_L5_avg', 'MP_L10_avg']].notna().mul([0.15, 0.25, 0.60]).sum(axis=1)    
    df['Early_Exit'] = ((df['MP_base'].notna()) & (df['MP_base'] > 0) &
                        (
                          ((df['MP'] - df['MP_base']) / df['MP_base'] <= -0.4) |  
                          ((df['MP_q4'] == 0) & (df['role'] == 1))
                        )).astype(int)    
    df = df[df.Early_Exit == 0]
    df = df.drop('Early_Exit', axis=1)
    
    return df

# Minutes Projection Model

In [187]:
def setup_df_mins(df, fltr_ee=True):

    df = df[['Season', 'Date', 'Team', 'Team_type', 'Opp', 'Player', 'Pos', 'role',
             'MP', 'MP_q4', 'Spread', 'team_game_num', 'is_OT']]    
    cleanup_cols = []
    cold_features = []
    if fltr_ee == True:
        df = filter_out_early_exits(df)
    
    df['team_mins_pct'] = df['MP'] / (240 + (df.is_OT * 25))
    for col in ['MP', 'team_mins_pct']:
        for N in [3, 5, 10]:
            df[f'{col}_L{N}_avg'] = (
                df.sort_values(['Player', 'Date']).groupby(['Player', 'Season'])[col].shift(1)
                 .rolling(window=N, min_periods=N)
                 .mean()
            )
            df[f'is_cold_{col}_L{N}'] = (df.groupby(['Player', 'Season']).cumcount() < N).astype(int)
            cold_features.append(f'is_cold_{col}_L{N}')
            cleanup_cols.append(f'{col}_L{N}_avg')
    df['MP_base'] = df[['MP_L3_avg', 'MP_L5_avg', 'MP_L10_avg']].mul([0.15, 0.25, 0.60]).sum(axis=1, skipna=True) / df[['MP_L3_avg', 'MP_L5_avg', 'MP_L10_avg']].notna().mul([0.15, 0.25, 0.60]).sum(axis=1)
    df['MP_tm_pct_base'] = df[['team_mins_pct_L3_avg', 'team_mins_pct_L5_avg', 'team_mins_pct_L10_avg']].mul([0.15, 0.25, 0.60]).sum(axis=1, skipna=True) / df[['team_mins_pct_L3_avg', 'team_mins_pct_L5_avg', 'team_mins_pct_L10_avg']].notna().mul([0.15, 0.25, 0.60]).sum(axis=1)
    
    df['role'] = np.where((df.role == 2) & (df.MP_base < 13), 3, df.role)
    
    games_last_14_days = df.sort_values(['Player', 'Season', 'Date']).groupby(['Player', 'Season']).rolling('14D', on='Date', closed='left')['MP'].count().reset_index().rename(columns={"MP": "gms_L14_days"})
    games_last_14_days = games_last_14_days.drop_duplicates(
        subset=['Player', 'Season', 'Date']
    )
    df = df.merge(games_last_14_days, on=['Player', 'Season', 'Date'])
    df['gms_L14_days'] = df.gms_L14_days.fillna(0).astype(int)    
    df['missed_games'] = (df.groupby(['Player', 'Team', 'Season'])['team_game_num'].diff().sub(1).fillna(0).astype(int))
    df['games_since_return'] = (df.groupby(['Player', 'Team', 'Season']).apply(
                                    lambda g: (
                                        (g['team_game_num'].diff().sub(1).fillna(0).gt(0))
                                        .cumsum()
                                        .groupby((g['team_game_num'].diff().sub(1).fillna(0).gt(0)).cumsum()).cumcount()
                                    )
                                ).reset_index(level=[0,1,2], drop=True))

    for N in [1, 3, 5]:
        df[f"recent_role_L{N}"] = (
            df.sort_values(['Player', 'Date']).groupby(['Player', 'Season'])['role'].shift(1)
              .transform(lambda x: x.rolling(N, min_periods=N)
                            .apply(lambda y: np.bincount(y.astype(np.int8), minlength=4).argmax(), raw=True))
        )
        df[f'is_cold_recent_role_L{N}'] = (df.groupby(['Player', 'Season']).cumcount() < N).astype(int)
        cold_features.append(f'is_cold_recent_role_L{N}')
        cleanup_cols.append(f"recent_role_L{N}")
    
#     df['game_spread_type'] = 0
#     df['game_spread_type'] = np.where(abs(df.Spread) < 13, 1, df.game_spread_type) 
#     df['game_spread_type'] = np.where((abs(df.Spread) >= 13) & (abs(df.Spread) <= 21), 2, df.game_spread_type) 
#     df['game_spread_type'] = np.where(abs(df.Spread) > 21, 3, df.game_spread_type) 
    
    df2 = create_df_missing(df, 'MP')
    df = df.merge(df2, on=["Season", "Date", "Team"], how='left')
    for col in ['starters_out', 'team_MP_available']:
        df[col] = df[col].fillna(0).astype(int)
        
    df['starters_returning'] = ((df['missed_games'] > 0) & (df['role'] == 1)).astype(int)
    df['returning_MP'] = (
        (df['MP_L10_avg'] * df['starters_returning'])
        .groupby([df['Team'], df['Date']])
        .transform('sum')
    )
    
    df['starters_returning'] = df.sort_values(['Team', 'Date']).groupby(['Team', 'Date'])['starters_returning'].transform('sum')
    df['team_MP_available'] = df['team_MP_available'] - df['returning_MP']

    df['MP_Change'] = 0
    MP_Inc_conds = (
#                     ((df.role != 3) & (df.starters_out > 2)) | 
                    ((df.role == 1) & (df.recent_role_L1 > 1.0)).astype(int) + 
                    (df.team_MP_available >= 110).astype(int)
                   )
    
    MP_Dec_conds = (
                    ((df.role > 1) & ((df.recent_role_L1 == 1.0))).astype(int) + 
                    (df.team_MP_available < -23).astype(int)
                   ) * -1
    df['MP_Change'] = MP_Inc_conds + MP_Dec_conds

    df['scenario_mins'] = (
        df.sort_values(['Season','Team','role','Pos','Date'])
          .groupby(['Season','Team','role','Pos'])['MP'].shift(1)
          .expanding()
          .mean() 
          .reset_index(drop=True)
    )
    
    df['MP_trend'] = df['MP_L3_avg'] - df['MP_L10_avg']
    df['Expected_MP'] = (
        (0.8 * df['scenario_mins']) +
        (df['team_MP_available'] * df['MP_tm_pct_base']) + 
        (0.2 * df['MP_base']) + df['MP_trend']
    )
    
    df["is_cold_start"] = (df[cold_features].eq(1).any(axis=1).astype(int))
    df['Team'] = df['Team'].astype('category')
    df['Opp'] = df['Opp'].astype('category')
    df['Player'] = df['Player'].astype('category')
    df['Pos'] = df['Pos'].astype('category')
    df = df.drop(['Team_type', 'team_game_num', 'is_OT', 'Spread', 'team_mins_pct', 'MP_tm_pct_base', 
                  'returning_MP', 'scenario_mins', 'MP_q4'] + cleanup_cols + cold_features, axis=1)    


    return df

In [188]:
df_mins = df.copy()
df_mins = setup_df_mins(df_mins)
display(df_mins)

game_dates = (df_mins[['Date']].drop_duplicates().sort_values('Date').reset_index(drop=True))
n_days = len(game_dates)
train_end = game_dates.loc[int(0.70 * n_days), 'Date']
val_end   = game_dates.loc[int(0.80 * n_days), 'Date']

mins_train_df = df_mins[df_mins['Date'] <= train_end]
mins_val_df   = df_mins[(df_mins['Date'] > train_end) & (df_mins['Date'] <= val_end)]
mins_test_df  = df_mins[df_mins['Date'] > val_end]
mins_DFS = (mins_train_df, mins_val_df, mins_test_df)

# Prev r2/mae/rmse best: 0.7064/4.0508/5.1790 [1/24/2026]
# mins_params = hyperparam_tuning(mins_DFS, "MP", n_iter=25)
# with open(f"{MDL_PATH}/mins_params.json", "w") as f:
#     json.dump(mins_params, f)

# Prev r2/mae/rmse best: 0.729/3.615/4.599 [1/24/2026]
mins_model, mins_results = refit_model(df_mins, 'MP', 'mins_params', min_train_days=910, rolling_window=180)
# feature_importance(mins_model, df_mins.columns.tolist())

mins_model.get_booster().save_model(f"{MDL_PATH}/mins_model.json")
print('Saved Mins booster!')

,Season,Date,Team,Opp,Player,Pos,role,MP,MP_base,gms_L14_days,missed_games,games_since_return,team_MP_available,starters_out,starters_returning,MP_Change,MP_trend,Expected_MP,is_cold_start
0,2021,2021-10-19,BRK,MIL,Blake Griffin,C,1,22.98,NaN,0,0,0,0.000,0,0,0,NaN,NaN,1
1,2021,2021-10-19,BRK,MIL,Bruce Brown,SF,2,3.75,NaN,0,0,0,0.000,0,0,0,NaN,NaN,1
2,2021,2021-10-19,BRK,MIL,Cam Thomas,SG,2,3.75,NaN,0,0,0,0.000,0,0,0,NaN,NaN,1
3,2021,2021-10-19,BRK,MIL,DeAndre' Bembry,SF,2,3.75,NaN,0,0,0,0.000,0,0,0,NaN,NaN,1
4,2021,2021-10-19,BRK,MIL,James Harden,PG,1,30.63,NaN,0,0,0,0.000,0,0,0,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106669,2025,2026-01-28,UTA,GSW,Jusuf Nurkic,C,1,22.63,31.0421,4,1,0,-43.036,1,2,-1,1.435667,22.015835,0
106670,2025,2026-01-28,UTA,GSW,Keyonte George,PG,1,32.70,36.3345,6,1,0,-43.036,1,2,-1,0.811667,21.511724,0
106671,2025,2026-01-28,UTA,GSW,Kyle Anderson,SF,2,21.88,20.7864,6,0,2,-43.036,1,2,-1,-2.340667,18.010012,0
106672,2025,2026-01-28,UTA,GSW,Kyle Filipowski,C,2,16.83,22.9553,7,0,6,-43.036,1,2,-1,-4.264667,16.115122,0


Rows: 106674, Dates: 945, min_train_days: 910
Progress:   2.86% (1/35)
Progress:   5.71% (2/35)
Progress:   8.57% (3/35)
Progress:  11.43% (4/35)
Progress:  14.29% (5/35)
Progress:  17.14% (6/35)
Progress:  20.00% (7/35)
Progress:  22.86% (8/35)
Progress:  25.71% (9/35)
Progress:  28.57% (10/35)
Progress:  31.43% (11/35)
Progress:  34.29% (12/35)
Progress:  37.14% (13/35)
Progress:  40.00% (14/35)
Progress:  42.86% (15/35)
Progress:  45.71% (16/35)
Progress:  48.57% (17/35)
Progress:  51.43% (18/35)
Progress:  54.29% (19/35)
Progress:  57.14% (20/35)
Progress:  60.00% (21/35)
Progress:  62.86% (22/35)
Progress:  65.71% (23/35)
Progress:  68.57% (24/35)
Progress:  71.43% (25/35)
Progress:  74.29% (26/35)
Progress:  77.14% (27/35)
Progress:  80.00% (28/35)
Progress:  82.86% (29/35)
Progress:  85.71% (30/35)
Progress:  88.57% (31/35)
Progress:  91.43% (32/35)
Progress:  94.29% (33/35)
Progress:  97.14% (34/35)
Progress: 100.00% (35/35)
Walk-forward RMSE: 4.567
Walk-forward MAE: 3.596
Walk

In [15]:
# analyze_df = mins_results.copy()
# analyze_df['Diff'] = analyze_df.Predictions - analyze_df.Actuals
# display(analyze_df[analyze_df.Date.isin(['2026-01-23'])].sort_values('Diff', ascending=False).head(10))

# plt.figure(figsize=(10,6))
# hist_col = 'Diff'
# plt.hist(analyze_df[hist_col], bins=30, color='skyblue', edgecolor='black')
# plt.title(f'Histogram of {hist_col}')
# plt.xlabel(hist_col)
# plt.ylabel('Frequency')
# plt.grid(axis='y', alpha=0.75)
# plt.show()

In [128]:
# df_lines = pd.read_csv(f"../tables/{YEAR}/parlay_lines.csv")
# df_lines['Date'] = pd.to_datetime(df_lines.Date)
# df_lines = df_lines[~(df_lines.Team.isnull())]

# df_lines["Team"] = team_encoder.transform(df_lines["Team"])
# df_pred = df_pred.merge(df_lines[['Date', 'Team', 'Spread', 'Total']], on=['Date', 'Team'], how='left')
# df_pred = df_pred[~df_pred[['Date', 'Team', 'Player']].duplicated(keep='last')]
# df_pred['Spread_x'] = np.where(df_pred.Spread_x.isnull(), df_pred.Spread_y, df_pred.Spread_x)
# df_pred['Total_x'] = np.where(df_pred.Total_x.isnull(), df_pred.Total_y, df_pred.Total_x)
# df_pred = df_pred.rename(columns={"Spread_x": "Spread", "Total_x": "Total"}).drop(['Spread_y', 'Total_y'], axis=1)
# df_prediction = df_pred.copy()

# # Predict Mins
# mins_booster = xgb.Booster()
# mins_booster.load_model("../ML_models/dev/mins_model.json")
# mins_model = XGBRegressor()
# mins_model._Booster = mins_booster

# df_prediction_mins = setup_df_mins(df_prediction)
# df_prediction_mins['MP_preds'] = mins_model.predict(df_prediction_mins.drop(['Season', 'Date', 'MP'], axis=1))
# df_prediction_mins = df_prediction_mins[df_prediction_mins.Date == now]

# df_prediction_mins['Team'] = team_encoder.inverse_transform(df_prediction_mins["Team"])
# df_prediction_mins['Opp'] = team_encoder.inverse_transform(df_prediction_mins["Opp"])
# df_prediction_mins['Player'] = player_encoder.inverse_transform(df_prediction_mins["Player"])

# if df_prediction_mins.shape[0] >= 50:
#     print(df_prediction_mins.shape[0], 'rows')
#     for tm in df_prediction_mins.Team.unique():
#         display(df_prediction_mins[df_prediction_mins.Team == tm])
# else:
#     display(df_prediction_mins)

# Stats Model

In [215]:
def setup_df_main(df, tgt_stat):
    
    df_prediction_mins = setup_df_mins(df, fltr_ee=False)
    mins_booster = xgb.Booster()
    mins_booster.load_model(f"{MDL_PATH}/mins_model.json")
    mins_model = XGBRegressor()
    mins_model._Booster = mins_booster

    df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'role', 'MP', 'MP_q4', 'team_game_num', 'is_OT', 
             'PTS', 'FG', 'FGA', 'FG%', 'TPA', 'TPM', 'TP%', 'FT', 'FTA', 'FT%', 'TOV', 'Spread', 'Total',
            'ORB']]
    cleanup_cols = []
    df['MP_proj'] = mins_model.predict(df_prediction_mins.drop(['Season', 'Date', 'MP'], axis=1))
    
    df['missed_games'] = (df.groupby(['Player', 'Team', 'Season'])['team_game_num'].diff().sub(1).fillna(0).astype(int))
    df['TeamPTS'] = (df.Total + (df.Spread * -1)) / 2
#     df['TeamPTS_type'] = 0
#     df['TeamPTS_type'] = np.where((df.TeamPTS < 110), 1, df.TeamPTS_type)
#     df['TeamPTS_type'] = np.where((df.TeamPTS >= 110) & (df.TeamPTS <= 130), 2, df.TeamPTS_type)
#     df['TeamPTS_type'] = np.where((df.TeamPTS > 130), 3, df.TeamPTS_type)

#     df[['pts_low', 'pts_mid', 'pts_high']] = pd.DataFrame({
#         'pts_low': ((110 - df.TeamPTS) / 20).clip(lower=0),
#         'pts_mid': (1 - abs(df.TeamPTS - 120) / 20).clip(lower=0),
#         'pts_high': ((df.TeamPTS - 130) / 20).clip(lower=0)
#     }, index=df.index)
#     w = df[['pts_low','pts_mid','pts_high']]
#     df[['pts_low','pts_mid','pts_high']] = w.div(w.sum(axis=1).replace(0,1), axis=0)
    
    
    # Create rolling + lag features
    df['eFG'] = (df['FG'] + 0.5 * df['TPM']) / df['FGA']
    df['TS']  = df['PTS'] / (2 * (df['FGA'] + 0.44 * df['FTA']))
    for col in [tgt_stat, 'FGA', 'FTA', 'ORB', 'TOV']:
        df[f'Team{col}'] = (df.sort_values(['Team', 'Date']).groupby(['Team', 'Date'])[col].transform('sum'))
        if col in [tgt_stat]:
            df[f'Team{col}_pct'] = df[col] / df[f'Team{col}']
            
    df['PlayerPace'] = np.where(df['MP'] > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
    df['TeamPace'] = ((df['TeamFGA'] + 0.44 * df['TeamFTA'] - df['TeamORB'] + df['TeamTOV']) / ((df['is_OT'] * 25) + 240))
    df['Player_Pace_Rel'] = df['PlayerPace'] / df['TeamPace']
    df['Pace_Minutes_Interaction'] = df['PlayerPace'] * df['MP']

    # Create rolling + lag features    
    for col in ['MP', tgt_stat, f'Team{tgt_stat}_pct', f'Def{tgt_stat}', 'FG', 'FGA', 'FT', 'FTA', 'Player_Pace_Rel', 'Pace_Minutes_Interaction']:
        for N in [1, 3, 5, 10]:
            if col == f'Def{tgt_stat}':
                df[f'Def{tgt_stat}_L{N}_avg'] = (
                    df[df.role <= 2]
                      .groupby(['Season', 'Date', 'Opp', 'Pos'])[tgt_stat]
                      .sum()
                      .groupby(['Opp', 'Pos', 'Season'])
                      .shift(1)
                      .rolling(window=N, min_periods=N)
                      .mean()
                      .reindex(df.set_index(['Season', 'Date', 'Opp', 'Pos']).index)
                      .values
                )
            else:
                df[f'{col}_L{N}_avg'] = (
                    df.sort_values(['Player', 'Date']).groupby(['Player', 'Season'])[col].shift(1)
                     .rolling(window=N, min_periods=N)
                     .mean()
                )
            cleanup_cols.append(f'{col}_L{N}_avg')
        df[f'{col}_base'] = df[[f'{col}_L3_avg', f'{col}_L5_avg', f'{col}_L10_avg']].mul([0.60, 0.25, 0.15]).sum(axis=1, skipna=True) / df[[f'{col}_L3_avg', f'{col}_L5_avg', f'{col}_L10_avg']].notna().mul([0.60, 0.25, 0.15]).sum(axis=1)    

    df2 = create_df_missing(df, tgt_stat)
    df = df.merge(df2, on=["Season", "Date", "Team"], how='left')
    for col in ['starters_out', f'team_{tgt_stat}_available']:
        df[col] = df[col].fillna(0)
    
    df['starters_returning'] = ((df['missed_games'] > 0) & (df['role'] == 1)).astype(int)
    df[f'returning_{tgt_stat}'] = (
        (df[f'{tgt_stat}_L10_avg'] * df['starters_returning'])
        .groupby([df['Team'], df['Date']])
        .transform('sum')
    )
    
    df['starters_returning'] = df.sort_values(['Team', 'Date']).groupby(['Team', 'Date'])['starters_returning'].transform('sum')
    df[f'team_{tgt_stat}_available'] = df[f'team_{tgt_stat}_available'] - df[f'returning_{tgt_stat}']
    df[f'Player_share_avail_{tgt_stat}'] = (df[f'team_{tgt_stat}_available'] * df[f'Team{tgt_stat}_pct_base'])
        
    df['Team'] = df['Team'].astype('category')
    df['Opp'] = df['Opp'].astype('category')
    df['Player'] = df['Player'].astype('category')
    df['Pos'] = df['Pos'].astype('category')
        
    df = df.drop(['team_game_num', 'is_OT', 'missed_games', 'MP', 'MP_q4', 'Spread', 'Total', 'role', 
                  f'Team{tgt_stat}', f'Team{tgt_stat}_pct', 'TeamFGA', 'TeamFTA', 'TeamORB', 'TeamTOV', 
                  'PlayerPace', 'TeamPace', 'Player_Pace_Rel', 'Pace_Minutes_Interaction',  
                 'FG', 'FGA', 'FG%', 'TPA', 'TPM', 'TP%', 'FT', 'FTA', 'FT%', 'TOV', 'eFG', 'TS', 'ORB', 
                 f'returning_{tgt_stat}', 'starters_out', 'starters_returning', f'team_{tgt_stat}_available'
                 ] + cleanup_cols, axis=1)

    return df

In [216]:
df_main = df.copy()
df_main = setup_df_main(df_main, 'PTS')
display(df_main)

game_dates = (df_main[['Date']].drop_duplicates().sort_values('Date').reset_index(drop=True))
n_days = len(game_dates)
train_end = game_dates.loc[int(0.70 * n_days), 'Date']
val_end   = game_dates.loc[int(0.80 * n_days), 'Date']

main_train_df = df_main[df_main['Date'] <= train_end]
main_val_df   = df_main[(df_main['Date'] > train_end) & (df_main['Date'] <= val_end)]
main_test_df  = df_main[df_main['Date'] > val_end]
main_DFS = (main_train_df, main_val_df, main_test_df)

# Prev r2/mae/rmse best: 0.6706/3.7633/5.0858 [1/24/2026]
# stat_params = hyperparam_tuning(main_DFS, 'PTS', n_iter=1, decay=0.99)
# with open(f"{MDL_PATH}/{tgt_stat}_params.json", "w") as f:
#     json.dump(stat_params, f)

# Prev r2/mae/rmse best: 0.695/3.628/4.840 [1/21/2026]
stat_model, stat_results = refit_model(df_main, 'PTS', f'{tgt_stat}_params', min_train_days=910, decay=0.99)
# feature_importance(stat_model, df_main.columns.tolist())

stat_model.get_booster().save_model(f"{MDL_PATH}/{tgt_stat}_model.json")
print(f"Saved {tgt_stat} booster!")

,Season,Date,Team,Opp,Player,Pos,PTS,MP_proj,MP_base,PTS_base,TeamPTS_pct_base,DefPTS_base,FG_base,FGA_base,FT_base,FTA_base,Player_Pace_Rel_base,Pace_Minutes_Interaction_base,Player_share_avail_PTS
0,2021,2021-10-19,BRK,MIL,Blake Griffin,C,6.0,24.448162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021,2021-10-19,BRK,MIL,Bruce Brown,SF,0.0,18.174227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,2021-10-19,BRK,MIL,Cam Thomas,SG,2.0,17.841209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,2021-10-19,BRK,MIL,DeAndre' Bembry,SF,0.0,14.692163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,2021-10-19,BRK,MIL,James Harden,PG,20.0,31.534994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127390,2025,2026-01-28,UTA,GSW,Kyle Anderson,SF,6.0,21.445969,19.73310,6.110,0.055807,19.580,2.40,4.255,0.980,1.465,0.540116,4.8996,-2.237873
127391,2025,2026-01-28,UTA,GSW,Kyle Filipowski,C,12.0,22.022057,21.06995,8.110,0.072600,25.055,3.17,5.455,1.065,1.725,0.688170,6.2140,-2.911279
127392,2025,2026-01-28,UTA,GSW,Lauri Markkanen,PF,18.0,36.104389,35.56625,26.660,0.229754,27.565,9.14,18.900,5.645,7.580,1.490188,22.2352,-9.213120
127393,2025,2026-01-28,UTA,GSW,Svi Mykhailiuk,SF,2.0,24.888882,19.06715,7.440,0.064623,19.580,2.77,4.320,1.220,1.235,0.571433,4.8634,-2.591402


Rows: 127395, Dates: 945, min_train_days: 910
Progress:   2.86% (1/35)
Progress:   5.71% (2/35)
Progress:   8.57% (3/35)
Progress:  11.43% (4/35)
Progress:  14.29% (5/35)
Progress:  17.14% (6/35)
Progress:  20.00% (7/35)
Progress:  22.86% (8/35)
Progress:  25.71% (9/35)
Progress:  28.57% (10/35)
Progress:  31.43% (11/35)
Progress:  34.29% (12/35)
Progress:  37.14% (13/35)
Progress:  40.00% (14/35)
Progress:  42.86% (15/35)
Progress:  45.71% (16/35)
Progress:  48.57% (17/35)
Progress:  51.43% (18/35)
Progress:  54.29% (19/35)
Progress:  57.14% (20/35)
Progress:  60.00% (21/35)
Progress:  62.86% (22/35)
Progress:  65.71% (23/35)
Progress:  68.57% (24/35)
Progress:  71.43% (25/35)
Progress:  74.29% (26/35)
Progress:  77.14% (27/35)
Progress:  80.00% (28/35)
Progress:  82.86% (29/35)
Progress:  85.71% (30/35)
Progress:  88.57% (31/35)
Progress:  91.43% (32/35)
Progress:  94.29% (33/35)
Progress:  97.14% (34/35)
Progress: 100.00% (35/35)
Walk-forward RMSE: 5.626
Walk-forward MAE: 4.329
Walk

In [125]:
# analyze_df = stat_results.copy()
# analyze_df['Diff'] = analyze_df.Predictions - analyze_df.Actuals
# display(analyze_df.sort_values('Diff', ascending=False).head(10))

# plt.figure(figsize=(10,6))
# hist_col = 'Diff'
# plt.hist(analyze_df[hist_col], bins=30, color='skyblue', edgecolor='black')
# plt.title(f'Histogram of {hist_col}')
# plt.xlabel(hist_col)
# plt.ylabel('Frequency')
# plt.grid(axis='y', alpha=0.75)
# plt.show()

# Residual PTS

In [16]:
def setup_df_res(df):
    
    df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'role', 'MP', 'MP_q4', 'team_game_num', 'Res_PTS', 'PTS_line',  
             'PTS', 'FG', 'FGA', 'FG%', 'TPA', 'TPM', 'TP%', 'FT', 'FTA', 'FT%', 'TOV', 
             'Spread', 'Total']]
    
    # Create rolling + lag features
    df3 = load_df('season_gamelogs')
    df3 = con.execute("""SELECT Date, Team, CAST(ROUND(SUM(MP), 0) as INT) as Team_Mins, 
                         CAST(SUM(FGA) as INT) as Team_FGA, CAST(SUM(FTA) as INT) as Team_FTA, CAST(SUM(TOV) as INT) as Team_TOV 
                         FROM df3
                         GROUP BY Date, Team""").fetchdf()
    df = df.merge(df3, on=['Date', 'Team'], how='left')
    df['eFG'] = (df['FG'] + 0.5 * df['TPM']) / df['FGA']
    df['TS']  = df['PTS'] / (2 * (df['FGA'] + 0.44 * df['FTA']))
    df['USG'] = (
        (df['FGA'] + 0.44*df['FTA'] + df['TOV']) * (df['Team_Mins'] / 5)
        / (df['MP'] * (df['Team_FGA'] + 0.44*df['Team_FTA'] + df['Team_TOV']))
    )

    LN_cols = []
    for col in ['MP', 'PTS', 'eFG', 'TS', 'USG']:
        for N in [3, 10]:
            if col != 'else':
                df[f'{col}_L{N}_avg'] = (
                    df.sort_values(['Player', 'Date']).groupby(['Player', 'Season'])[col].shift(1)
                     .rolling(window=N, min_periods=N)
                     .mean()
                )

                if col not in ['MP']:
                    LN_cols.append(f'{col}_L{N}_avg')
        df[f'{col}_trend'] = df[f'{col}_L3_avg'] - df[f'{col}_L10_avg']
    
    df['game_spread_type'] = 0
    df['game_spread_type'] = np.where(abs(df.Spread) < 13, 1, df.game_spread_type) 
    df['game_spread_type'] = np.where((abs(df.Spread) >= 13) & (abs(df.Spread) <= 18), 2, df.game_spread_type) 
    df['game_spread_type'] = np.where(abs(df.Spread) > 18, 3, df.game_spread_type) 
    
    df['TeamPTS'] = (df.Total + (df.Spread * -1)) / 2
#     df['TeamPTS_type'] = 0
#     df['TeamPTS_type'] = np.where((df.TeamPTS < 110), 1, df.TeamPTS_type)
#     df['TeamPTS_type'] = np.where((df.TeamPTS >= 110) & (df.TeamPTS <= 130), 2, df.TeamPTS_type)
#     df['TeamPTS_type'] = np.where((df.TeamPTS > 130), 3, df.TeamPTS_type)

    df[['pts_low', 'pts_mid', 'pts_high']] = pd.DataFrame({
        'pts_low': ((110 - df.TeamPTS) / 20).clip(lower=0),
        'pts_mid': (1 - abs(df.TeamPTS - 120) / 20).clip(lower=0),
        'pts_high': ((df.TeamPTS - 130) / 20).clip(lower=0)
    }, index=df.index)
#     w = df[['pts_low','pts_mid','pts_high']]
#     df[['pts_low','pts_mid','pts_high']] = w.div(w.sum(axis=1).replace(0,1), axis=0)
        
    for col in ['TeamPTS']:
        for N in [3, 10]:
            df[f'{col}_L{N}_avg'] = (
                df.sort_values(['Player', 'Date']).groupby(['Player', 'Season'])[col].shift(1)
                  .rolling(window=N, min_periods=N)
                  .mean()
            )
            df[f'PTS_pct_L{N}'] = df[f'PTS_L{N}_avg'] / df[f'TeamPTS_L{N}_avg']
            df = df.drop(f'TeamPTS_L{N}_avg', axis=1)
        df['PTS_pct_trend'] = df['PTS_pct_L3'] - df['PTS_pct_L10']
        df = df.drop(['PTS_pct_L3', 'PTS_pct_L10'], axis=1)
    
    df2 = create_df_missing(df, 'PTS')
    df = df.merge(df2, on=["Season", "Date", "Team"], how='left')
    for col in ['starters_out', 'team_PTS_available']:
        df[col] = df[col].fillna(0)
    df['starters_out_L1'] = (
        df.sort_values(['Player', 'Date']).groupby(['Player', 'Season'])['starters_out'].shift(1)
          .rolling(window=1, min_periods=1)
          .mean()
    )
    df['starters_returning'] = np.where(df['starters_out_L1'] > df['starters_out'], df['starters_out_L1'] - df['starters_out'], 0)

    df['Team'] = df['Team'].astype('category')
    df['Opp'] = df['Opp'].astype('category')
    df['Player'] = df['Player'].astype('category')
    df['Pos'] = df['Pos'].astype('category')
    df = df.drop(['team_game_num', 'Spread', 'Total', 'MP_q4', 'TeamPTS', 'Team_FGA', 'Team_FTA', 'Team_TOV', 
                 'PTS', 'FG', 'FGA', 'FG%', 'TPA', 'TPM', 'TP%', 'FT', 'FTA', 'FT%', 'TOV', 'eFG', 'TS', 'USG', 
                  'Team_Mins', 'starters_out_L1'] + LN_cols, axis=1)
        
    return df

##### Regressor

In [21]:
df_res = df[(~df.PTS_line.isnull())].copy()
df_res = setup_df_res(df_res)
display(df_res)

game_dates = (df_res[['Date']].drop_duplicates().sort_values('Date').reset_index(drop=True))
n_days = len(game_dates)
train_end = game_dates.loc[int(0.70 * n_days), 'Date']
val_end   = game_dates.loc[int(0.80 * n_days), 'Date']

res_train_df = df_res[df_res['Date'] <= train_end]
res_val_df   = df_res[(df_res['Date'] > train_end) & (df_res['Date'] <= val_end)]
res_test_df  = df_res[df_res['Date'] > val_end]
res_DFS = (res_train_df, res_val_df, res_test_df)

# Prev r2/mae/rmse best: 0.2124/4.3069/5.6478 [1/24/2026]
# res_params = hyperparam_tuning(res_DFS, 'Res_PTS', n_iter=25)
# with open(f"{MDL_PATH}/Res_PTS_RG_params.json", "w") as f:
#     json.dump(res_params, f)

# Prev mae best: 4.1101 [1/18/2026]
res_model, res_results = refit_model(df_res, 'Res_PTS', 'Res_PTS_RG_params', min_train_days=50)
# feature_importance(res_model, df_res.columns.tolist())

res_model.get_booster().save_model(f"{MDL_PATH}/Res_PTS_RG_model.json")
print("Saved Res_PTS_RG booster!")

,Season,Date,Team,Opp,Player,Pos,role,MP,Res_PTS,PTS_line,MP_L3_avg,MP_L10_avg,MP_trend,PTS_trend,eFG_trend,TS_trend,USG_trend,game_spread_type,pts_low,pts_mid,pts_high,PTS_pct_trend,team_PTS_available,starters_out,starters_returning
0,2025,2025-11-20,ATL,SAS,Dyson Daniels,SG,1,30.68,-3.5,11.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0.7,0.0,NaN,0.0,0.0,0.0
1,2025,2025-11-20,ATL,SAS,Jalen Johnson,SF,1,37.95,3.5,22.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0.7,0.0,NaN,0.0,0.0,0.0
2,2025,2025-11-20,ATL,SAS,Kristaps Porzingis,C,1,29.05,-0.5,16.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0.7,0.0,NaN,0.0,0.0,0.0
3,2025,2025-11-20,ATL,SAS,Nickeil Alexander-Walker,SG,1,33.65,20.5,17.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0.7,0.0,NaN,0.0,0.0,0.0
4,2025,2025-11-20,ATL,SAS,Onyeka Okongwu,C,2,25.13,0.5,14.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0.7,0.0,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5754,2025,2026-01-26,POR,BOS,Jrue Holiday,PG,1,24.45,1.5,12.5,20.283333,NaN,NaN,NaN,NaN,NaN,NaN,1,0.8,0.0,0.0,NaN,35.0,2.0,0.0
5755,2025,2026-01-26,POR,BOS,Rayan Rupert,SG,2,15.80,0.5,3.5,19.746667,NaN,NaN,NaN,NaN,NaN,NaN,1,0.8,0.0,0.0,NaN,35.0,2.0,0.0
5756,2025,2026-01-26,POR,BOS,Robert Williams,C,2,19.68,-0.5,6.5,18.656667,16.084,2.572667,-0.400000,NaN,NaN,-0.021068,1,0.8,0.0,0.0,-0.009271,35.0,2.0,0.0
5757,2025,2026-01-26,POR,BOS,Shaedon Sharpe,SG,1,34.32,-14.5,23.5,33.440000,32.221,1.219000,1.500000,-0.029100,-0.031318,0.035742,1,0.8,0.0,0.0,0.015482,35.0,2.0,0.0


Rows: 5759, Dates: 66, min_train_days: 50
Progress:   6.25% (1/16)
Progress:  12.50% (2/16)
Progress:  18.75% (3/16)
Progress:  25.00% (4/16)
Progress:  31.25% (5/16)
Progress:  37.50% (6/16)
Progress:  43.75% (7/16)
Progress:  50.00% (8/16)
Progress:  56.25% (9/16)
Progress:  62.50% (10/16)
Progress:  68.75% (11/16)
Progress:  75.00% (12/16)
Progress:  81.25% (13/16)
Progress:  87.50% (14/16)
Progress:  93.75% (15/16)
Progress: 100.00% (16/16)
Walk-forward MAE: 4.229058576918787
|Pred| >= 0: accuracy = 0.631, n = 1612
|Pred| >= 1: accuracy = 0.678, n = 1142
|Pred| >= 2: accuracy = 0.735, n = 740
|Pred| >= 3: accuracy = 0.802, n = 449
Saved Res_PTS_RG booster!


##### Classifier

In [22]:
df_res = df[(~df.PTS_line.isnull())].copy()
df_res = setup_df_res(df_res)
df_res['Bet'] = (df_res['Res_PTS'] > 0).astype(int)  # 1 = over, 0 = under
df_res = df_res.drop('Res_PTS', axis=1)
# display(df_res)

game_dates = (df_res[['Date']].drop_duplicates().sort_values('Date').reset_index(drop=True))
n_days = len(game_dates)
train_cut = int(0.70 * n_days)
val_cut   = int(0.80 * n_days)
train_end = game_dates.loc[train_cut, 'Date']
val_end   = game_dates.loc[val_cut, 'Date']

res_train_df = df_res[df_res['Date'] <= train_end]
res_val_df   = df_res[(df_res['Date'] > train_end) & (df_res['Date'] <= val_end)]
res_test_df  = df_res[df_res['Date'] > val_end]
res_DFS = (res_train_df, res_val_df, res_test_df)

# Test Accuracy: 0.6672 [1/24/2026]
# res_params = hyperparam_tuning(res_DFS, 'Bet', is_classification=True, n_iter=25)
# with open(f"{MDL_PATH}/Res_PTS_CLF_params.json", "w") as f:
#     json.dump(res_params, f)

# Prev roc_auc best: 0.720 [1/18/2026]
res_model, res_results = refit_model(df_res, 'Bet', 'Res_PTS_RG_params', min_train_days=50)
# feature_importance(res_model, df_res.columns.tolist())

res_model.get_booster().save_model(f"{MDL_PATH}/Res_PTS_CLF_model.json")
print("Saved Res_PTS_CLF booster!")

Rows: 5759, Dates: 66, min_train_days: 50
Progress:   6.25% (1/16)
Progress:  12.50% (2/16)
Progress:  18.75% (3/16)
Progress:  25.00% (4/16)
Progress:  31.25% (5/16)
Progress:  37.50% (6/16)
Progress:  43.75% (7/16)
Progress:  50.00% (8/16)
Progress:  56.25% (9/16)
Progress:  62.50% (10/16)
Progress:  68.75% (11/16)
Progress:  75.00% (12/16)
Progress:  81.25% (13/16)
Progress:  87.50% (14/16)
Progress:  93.75% (15/16)
Progress: 100.00% (16/16)
Confusion Matrix:
 [[548 286]
 [300 478]]
Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.66      0.65       834
           1       0.63      0.61      0.62       778

    accuracy                           0.64      1612
   macro avg       0.64      0.64      0.64      1612
weighted avg       0.64      0.64      0.64      1612

ROC AUC: 0.693
High-confidence hit rate (<= 0.3 & >= 0.7): 0.77
Saved Res_PTS_CLF booster!


# HT Stats

In [311]:
def setup_df_ht(df):
    
    df_full = setup_df_main(df, 'PTS')
    stat_booster = xgb.Booster()
    stat_booster.load_model(f"{MDL_PATH}/PTS_model.json")
    stat_model = XGBRegressor()
    stat_model._Booster = stat_booster
    
    df_prediction_mins = setup_df_mins(df, fltr_ee=False)
    mins_booster = xgb.Booster()
    mins_booster.load_model(f"{MDL_PATH}/mins_model.json")
    mins_model = XGBRegressor()
    mins_model._Booster = mins_booster

    df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 
             'MP_h1', 'PTS_h1', 'PTS_h2', 'FG_h1', 'FGA_h1', 'FGA_h2', 'FT_h1', 'FTA_h1', 'TPM_h1', 'TPA_h1', 'PF_h1', 
             'ORB_h1', 'TOV_h1', 'PTS', 'FGA']]
    df["PTS_proj"] = stat_model.predict(df_full.drop(['Season', 'Date', 'PTS'], axis=1))
    df['MP_proj'] = mins_model.predict(df_prediction_mins.drop(['Season', 'Date', 'MP'], axis=1))

    cleanup_cols = []
    df['MP_proj_pct'] = np.where(df['MP_proj'] > 0, (df['MP_proj'] - df['MP_h1']) / df['MP_proj'], np.nan)
    df['PTS_proj_pct'] = np.where(df['PTS_proj'] > 0, (df['PTS_h1'] - df['PTS_proj']) / df['PTS_proj'], np.nan)

    
    for col in ['PTS', 'FGA', 'FTA', 'ORB', 'TOV']:
        df[f'Team{col}_h1'] = (df.sort_values(['Team', 'Date']).groupby(['Team', 'Date'])[f'{col}_h1'].transform('sum'))
        if col not in ['FTA', 'ORB', 'TOV']:
            df[f'Team{col}_pct_h1'] = df[f'{col}_h1'] / df[f'Team{col}_h1']
    
    df['OppTeamPTS_h1'] = (
        (
        df.groupby(['Season', 'Date', 'Team'], as_index=True)['TeamPTS_h1']
        .first()
        ).reindex(
        pd.MultiIndex.from_frame(df[['Season', 'Date', 'Opp']])
        ).to_numpy())
    df['Spread_h1'] = df['TeamPTS_h1'] - df['OppTeamPTS_h1']

    df['Player_Pace'] = np.where(df['MP_h1'] > 0, (df['FGA_h1'] + 0.44 * df['FTA_h1']) / df['MP_h1'], 0)
    df['Team_Pace'] = ((df['TeamFGA_h1'] + 0.44 * df['TeamFTA_h1'] - df['TeamORB_h1'] + df['TeamTOV_h1']) / 120)
    df['Player_Pace_Rel'] = df['Player_Pace'] / df['Team_Pace']
    df['Pace_Minutes_Interaction'] = df['Player_Pace'] * df['MP_h1']

    # Create rolling + lag features    
    for col in ['MP', 'PTS', 'FG_h1', 'FGA_h1', 'FGA_h2', 'PTS_h1', 'PTS_h2']:
        for N in [1, 3, 5, 10]:
            df[f'{col}_L{N}_avg'] = (
                df.sort_values(['Player', 'Date']).groupby(['Player', 'Season'])[col].shift(1)
                 .rolling(window=N, min_periods=N)
                 .mean()
            )
            cleanup_cols.append(f'{col}_L{N}_avg')
        df[f'{col}_base'] = df[[f'{col}_L3_avg', f'{col}_L5_avg', f'{col}_L10_avg']].mul([0.15, 0.25, 0.60]).sum(axis=1, skipna=True) / df[[f'{col}_L3_avg', f'{col}_L5_avg', f'{col}_L10_avg']].notna().mul([0.15, 0.25, 0.60]).sum(axis=1)
        
    df['Team'] = df['Team'].astype('category')
    df['Opp'] = df['Opp'].astype('category')
    df['Player'] = df['Player'].astype('category')
    df['Pos'] = df['Pos'].astype('category')    
    df = df.drop(['TeamFGA_h1', 'TeamPTS_h1', 'OppTeamPTS_h1', 'TeamORB_h1', 'TeamTOV_h1', 'TeamFTA_h1', 'Player_Pace', 'Team_Pace', 
                  'MP_base', 'PTS_base', 'MP', 'PTS_h2', 'FGA', 'FGA_h2', 'ORB_h1', 'TOV_h1'] + cleanup_cols, axis=1)

    return df

In [310]:
df_ht = df.copy()
df_ht = setup_df_ht(df_ht)
display(df_ht)

game_dates = (df_ht[['Date']].drop_duplicates().sort_values('Date').reset_index(drop=True))
n_days = len(game_dates)
train_end = game_dates.loc[int(0.70 * n_days), 'Date']
val_end   = game_dates.loc[int(0.80 * n_days), 'Date']

ht_train_df = df_ht[df_ht['Date'] <= train_end]
ht_val_df   = df_ht[(df_ht['Date'] > train_end) & (df_ht['Date'] <= val_end)]
ht_test_df  = df_ht[df_ht['Date'] > val_end]
ht_DFS = (ht_train_df, ht_val_df, ht_test_df)

# 0.786 / 0.702 / 1.012 [1/29/26]
ht_train_dict = {
                 'mean': {'quantile': False, 'q_val': 0.5}, 
                 'Qlow': {'quantile': True, 'q_val': 0.15}, 
                 'Qhigh': {'quantile': True, 'q_val': 0.85}
                }
for key in ht_train_dict.keys():
#     ht_params = hyperparam_tuning(ht_DFS, 'PTS', n_iter=1, decay=0.99, quantile=ht_train_dict[key]['quantile'], q_val=ht_train_dict[key]['q_val'])
#     with open(f"{MDL_PATH}/ht_PTS_{key}_params.json", "w") as f:
#         json.dump(ht_params, f)

    ht_model, ht_results = refit_model(df_ht, 'PTS', f'ht_PTS_{key}_params', min_train_days=910, decay=0.99)
#     feature_importance(ht_model, df_ht.columns.tolist())

    ht_model.get_booster().save_model(f"{MDL_PATH}/ht_PTS_{key}_model.json")
    print(f"Saved ht_PTS_{key} booster!")

,Season,Date,Team,Opp,Player,Pos,MP_h1,PTS_h1,FG_h1,FGA_h1,FT_h1,FTA_h1,TPM_h1,TPA_h1,PF_h1,PTS,PTS_proj,MP_proj,MP_proj_pct,PTS_proj_pct,TeamPTS_pct_h1,TeamFGA_pct_h1,Spread_h1,Player_Pace_Rel,Pace_Minutes_Interaction,FG_h1_base,FGA_h1_base,FGA_h2_base,PTS_h1_base,PTS_h2_base
1,2021,2021-10-19,BRK,MIL,Blake Griffin,C,11.98,6.0,2.0,3.0,2.0,2.0,0.0,0.0,0.0,6.0,8.547970,24.448162,0.509984,0.298079,0.101695,0.073171,-7.0,0.799029,3.88,NaN,NaN,NaN,NaN,NaN
2,2021,2021-10-19,BRK,MIL,Bruce Brown,SF,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.377724,18.174227,1.000000,1.000000,0.000000,0.000000,-7.0,0.000000,0.00,NaN,NaN,NaN,NaN,NaN
3,2021,2021-10-19,BRK,MIL,Cam Thomas,SG,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,13.538774,17.841209,1.000000,1.000000,0.000000,0.000000,-7.0,0.000000,0.00,NaN,NaN,NaN,NaN,NaN
6,2021,2021-10-19,BRK,MIL,DeAndre' Bembry,SF,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.907993,14.692163,1.000000,1.000000,0.000000,0.000000,-7.0,0.000000,0.00,NaN,NaN,NaN,NaN,NaN
9,2021,2021-10-19,BRK,MIL,James Harden,PG,17.83,15.0,5.0,9.0,2.0,2.0,3.0,6.0,2.0,20.0,24.915615,31.534994,0.434596,0.397968,0.254237,0.219512,-7.0,1.367078,9.88,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256766,2025,2026-01-28,UTA,GSW,Kyle Anderson,SF,10.35,4.0,2.0,4.0,0.0,0.0,0.0,0.0,2.0,6.0,6.870486,21.445969,0.517392,0.417800,0.072727,0.086957,-13.0,0.818800,4.00,1.13,2.32,2.85,2.49,4.55
256767,2025,2026-01-28,UTA,GSW,Kyle Filipowski,C,10.35,10.0,4.0,4.0,0.0,0.0,2.0,2.0,1.0,12.0,7.416295,22.022057,0.530017,-0.348382,0.181818,0.086957,-13.0,0.818800,4.00,1.87,3.77,3.05,4.85,4.19
256768,2025,2026-01-28,UTA,GSW,Lauri Markkanen,PF,16.55,6.0,2.0,8.0,2.0,2.0,0.0,3.0,0.0,18.0,27.843861,36.104389,0.541607,0.784513,0.109091,0.173913,-13.0,1.136771,8.88,4.63,10.16,8.52,13.58,13.08
256770,2025,2026-01-28,UTA,GSW,Svi Mykhailiuk,SF,11.33,0.0,0.0,2.0,0.0,0.0,0.0,2.0,1.0,2.0,7.664839,24.888882,0.544777,1.000000,0.000000,0.043478,-13.0,0.373988,2.00,1.55,3.03,2.19,4.41,3.30



Trial 1/1: {'n_estimators': 1419, 'learning_rate': np.float64(0.05369808015563413), 'max_depth': 3, 'min_child_weight': 7, 'subsample': np.float64(0.8347738413359717), 'colsample_bytree': np.float64(0.910764403894077), 'gamma': np.float64(0.4923313235640901), 'reg_lambda': np.float64(2.663225495983999), 'reg_alpha': np.float64(0.07880968376762765)}
Validation MAE: 3.1666

Best score: 3.166620447910881
Best parameters: {'learning_rate': np.float64(0.05369808015563413), 'max_depth': 3, 'min_child_weight': 7, 'subsample': np.float64(0.8347738413359717), 'colsample_bytree': np.float64(0.910764403894077), 'gamma': np.float64(0.4923313235640901), 'reg_lambda': np.float64(2.663225495983999), 'reg_alpha': np.float64(0.07880968376762765), 'objective': 'reg:squarederror', 'enable_categorical': True, 'tree_method': 'hist', 'device': 'cuda', 'seed': 42}

Test Metrics:
RMSE: 4.331840859586777
MAE: 3.2586635351981226
R²: 0.7601636657107816
Rows: 127395, Dates: 945, min_train_days: 910
Progress:   2

In [70]:
# analyze_df = ht_results.copy()
# analyze_df['Diff'] = analyze_df.Predictions - analyze_df.Actuals
# display(analyze_df.sort_values('Diff', ascending=True).head(10))

# plt.figure(figsize=(10,6))
# hist_col = 'Diff'
# plt.hist(analyze_df[hist_col], bins=30, color='skyblue', edgecolor='black')
# plt.title(f'Histogram of {hist_col}')
# plt.xlabel(hist_col)
# plt.ylabel('Frequency')
# plt.grid(axis='y', alpha=0.75)
# plt.show()

In [312]:
df_ht = setup_df_ht(df_pred)
df_ht = df_ht[df_ht.Date == now]
display(df_ht)
partition_save_df(df_ht, f"../tables/{YEAR}/ht_api_input.csv")

,Season,Date,Team,Opp,Player,Pos,MP_h1,PTS_h1,FG_h1,FGA_h1,FT_h1,FTA_h1,TPM_h1,TPA_h1,PF_h1,PTS,PTS_proj,MP_proj,MP_proj_pct,PTS_proj_pct,TeamPTS_pct_h1,TeamFGA_pct_h1,Spread_h1,Player_Pace_Rel,Pace_Minutes_Interaction,FG_h1_base,FGA_h1_base,FGA_h2_base,PTS_h1_base,PTS_h2_base
185085,2025,2026-01-29,ATL,HOU,Asa Newell,PF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.128745,12.868668,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.770,1.670000,2.150000,1.870,2.760000
185094,2025,2026-01-29,ATL,HOU,CJ McCollum,PG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.096800,26.328728,NaN,NaN,NaN,NaN,0.0,NaN,NaN,4.750,8.000000,6.125000,11.750,6.500000
185103,2025,2026-01-29,ATL,HOU,Caleb Houstan,SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.448093,10.408571,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.120,0.340000,1.110000,0.480,1.650000
185112,2025,2026-01-29,ATL,HOU,Christian Koloko,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.274672,25.624039,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.000,1.666667,1.333333,2.000,2.666667
185121,2025,2026-01-29,ATL,HOU,Corey Kispert,SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.114856,25.072842,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.875,4.250000,3.125000,4.625,3.625000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187085,2025,2026-01-29,WAS,MIL,Skal Labissiere,PF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.856599,14.871835,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187090,2025,2026-01-29,WAS,MIL,Trae Young,PG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.032689,19.118029,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187095,2025,2026-01-29,WAS,MIL,Tre Johnson,SG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.223442,30.973494,NaN,NaN,NaN,NaN,0.0,NaN,NaN,2.790,6.980000,7.000000,8.580,8.330000
187100,2025,2026-01-29,WAS,MIL,Tristan Vukcevic,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.367995,10.751883,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.020,2.800000,3.120000,2.760,4.780000


../tables/2025/ht_api_input.csv saved!


# Today's predictions

In [147]:
# df_yesterday = pd.read_csv(f'../tables/{YEAR}/gmday_preds_PTS.csv')
# df_yesterday['Date'] = pd.to_datetime(df_yesterday.Date)
# df_yesterday = df_yesterday[(df_yesterday.Date == (datetime.strptime(now, "%Y-%m-%d") - timedelta(days=1)).strftime("%Y-%m-%d"))]\
#                 .rename(columns={"MP": "MP_proj"})

# df_gms = pd.read_csv(f"../tables/{YEAR}/season_gamelogs.csv")
# df_gms['Date'] = pd.to_datetime(df_gms.Date)
# df_lines = pd.read_csv(f"../tables/{YEAR}/parlay_lines.csv")
# df_lines['Date'] = pd.to_datetime(df_lines.Date)
# df_lines = df_lines[~(df_lines.Team.isnull()) & ~(df_lines.PTS_line.isnull())].drop(['Pos', 'Spread', 'Total'], axis=1)
# df_gms = df_gms.merge(df_lines, on=['Date', 'Team', 'Player'])
# df_gms['Res_PTS'] = df_gms.PTS - df_gms.PTS_line

# df_yesterday = df_yesterday.merge(df_gms[['Date', 'Team', 'Player', 'PTS', 'Res_PTS', 'MP']], on=['Date', 'Team', 'Player'])
# df_yesterday = df_yesterday[['Date', 'Team', 'Player', 'MP', 'MP_proj', 'PTS_line', 'PTS_proj', 'PTS', 'Res_PTS_proj', 'Res_PTS', 'pred_prob', 'pred_class']][df_yesterday.MP > 0]

# # Mins
# df_yesterday['Diff'] = abs(df_yesterday['MP_proj'] - df_yesterday['MP'])
# df_yesterday['InTgtRange'] = np.where(df_yesterday['Diff'] <= 3, 1, 0)
# print("\nYesterday's Results:")
# print("Total Accuracy (Minutes-in-range):", (df_yesterday.InTgtRange == 1).mean())
# print((df_yesterday.InTgtRange == 1).sum(), '/', df_yesterday.shape[0])
# df_yesterday = df_yesterday.drop(['Diff', 'InTgtRange'], axis=1)

# # Raw PTS
# df_yesterday['Diff'] = abs(df_yesterday['PTS'] - df_yesterday['PTS_proj'])
# df_yesterday['InTgtRange'] = np.where(df_yesterday['Diff'] <= 3, 1, 0)
# df_yesterday['Act_Res'] = np.where((df_yesterday.PTS > df_yesterday.PTS_line), 1, 0)
# df_yesterday['Pred_Res'] = np.where((df_yesterday.PTS_proj > df_yesterday.PTS_line), 1, 0)
# df_yesterday['PHit1'] = np.where(df_yesterday['Act_Res'] == df_yesterday['Pred_Res'], 1, 0)
# print("Total Accuracy (Raw PTS):", (df_yesterday.PHit1 == 1).mean())
# print((df_yesterday.PHit1 == 1).sum(), "/", df_yesterday.shape[0])
# print("Total Accuracy (Raw PTS-in-range):", (df_yesterday.InTgtRange == 1).mean())
# df_yesterday = df_yesterday.drop(['Diff', 'InTgtRange', 'Act_Res', 'Pred_Res'], axis=1)

# # Res PTS (Regression)
# df_yesterday['Act_Res'] = np.where(df_yesterday['Res_PTS'] > 0, 'O', 'U')
# df_yesterday['Pred_Res'] = np.where(df_yesterday['Res_PTS_proj'] > 0, 'O', 'U')
# df_yesterday['PHit2'] = np.where(df_yesterday['Act_Res'] == df_yesterday['Pred_Res'], 1, 0)
# print("Total Accuracy (ResPTS Regression):", (df_yesterday.PHit2 == 1).mean())
# print((df_yesterday.PHit2 == 1).sum(), "/", df_yesterday.shape[0])
# df_yesterday = df_yesterday.drop(['Act_Res', 'Pred_Res'], axis=1)

# # Res PTS (Classifier)
# df_yesterday['Act_Res'] = np.where(df_yesterday['Res_PTS'] > 0, 1, 0)
# df_yesterday['PHit3'] = np.where(df_yesterday['Act_Res'] == df_yesterday['pred_class'], 1, 0)
# df_yesterday['pred_class'] = np.where(df_yesterday['pred_class'] == 1, 'O', 'U')
# print("Total Accuracy (ResPTS Classification):", (df_yesterday.PHit3 == 1).mean())
# print((df_yesterday.PHit3 == 1).sum(), "/", df_yesterday.shape[0])
# df_yesterday = df_yesterday.drop(['Act_Res'], axis=1)

# df_yesterday['Majority'] = np.where(((df_yesterday.PTS_proj > df_yesterday.PTS_line).astype(int) + (df_yesterday['Res_PTS_proj'] > 0).astype(int) + (df_yesterday['pred_class'] == 1).astype(int)) >= 2, 1, 0)
# df_yesterday['MajorityHit'] = np.where((df_yesterday.Majority == 1) & (df_yesterday.PTS > df_yesterday.PTS_line), 1, 0)
# df_yesterday['MajorityHit'] = np.where((df_yesterday.Majority == 0) & (df_yesterday.PTS < df_yesterday.PTS_line), 1, df_yesterday.MajorityHit)
# print("Total Accuracy (MajorityHit):", (df_yesterday.MajorityHit == 1).mean())

# df_yesterday['AllAgree'] = '-'
# df_yesterday['AllAgree'] = np.where((df_yesterday.PHit1 == 1) & (df_yesterday.PHit2 == 1) & (df_yesterday.PHit3 == 1), 1, df_yesterday['AllAgree'])
# df_yesterday['AllAgree'] = np.where((df_yesterday.PHit1 == 0) & (df_yesterday.PHit2 == 0) & (df_yesterday.PHit3 == 0), 0, df_yesterday['AllAgree'])
# print("Total Accuracy (AllAgree):", ((df_yesterday.AllAgree == 1).sum() / ((df_yesterday.AllAgree == 0).sum() + (df_yesterday.AllAgree == 1).sum())))

# df_yesterday = df_yesterday.drop(['Majority', 'MajorityHit', 'AllAgree'], axis=1).sort_values('PTS_line', ascending=False)

# # if df_yesterday.shape[0] >= 50:
# #     for tm in df_yesterday.Team.unique():
# #         display(df_yesterday[(df_yesterday.Team == tm)]) #  & (df_yesterday.PHit == 1)
# # else:
# #     display(df_yesterday)

In [322]:
df_lines = pd.read_csv(f"../tables/{YEAR}/parlay_lines.csv")
df_lines['Date'] = pd.to_datetime(df_lines.Date)
df_lines = df_lines[~(df_lines.Team.isnull())]

df_pred = df_pred.merge(df_lines[['Date', 'Team', 'Spread', 'Total']], on=['Date', 'Team'], how='left')
df_pred = df_pred[~df_pred[['Date', 'Team', 'Player']].duplicated(keep='last')]
df_pred['Spread_x'] = np.where(df_pred.Spread_x.isnull(), df_pred.Spread_y, df_pred.Spread_x)
df_pred['Total_x'] = np.where(df_pred.Total_x.isnull(), df_pred.Total_y, df_pred.Total_x)
df_pred = df_pred.rename(columns={"Spread_x": "Spread", "Total_x": "Total"}).drop(['Spread_y', 'Total_y'], axis=1)
df_prediction = df_pred.copy()

# Predict Stat
stat_booster = xgb.Booster()
stat_booster.load_model(f"{MDL_PATH}/{tgt_stat}_model.json")
stat_model = XGBRegressor()
stat_model._Booster = stat_booster
res_booster_RG = xgb.Booster()
res_booster_RG.load_model(f"{MDL_PATH}/Res_PTS_RG_model.json")
res_model_RG = XGBRegressor()
res_model_RG._Booster = res_booster_RG
res_model_CLF = XGBClassifier()
res_model_CLF.load_model(f"{MDL_PATH}/Res_PTS_CLF_model.json")

df_prediction = setup_df_main(df_prediction, 'PTS')
feature_cols = [col for col in df_prediction.columns if col not in ['Season', 'Date', 'MP_preds', 'PTS']]
df_prediction = df_prediction[df_prediction.Date == now][feature_cols]
df_prediction["PTS_proj"] = stat_model.predict(df_prediction)

df_prediction2 = df_pred[(~df_pred.PTS_line.isnull())].copy()
df_prediction2 = setup_df_res(df_prediction2)
df_prediction2 = df_prediction2.merge(df_prediction_mins[['Date', 'Team', 'Player', 'MP_preds']], on=['Date', 'Team', 'Player'], how='left')
df_prediction2['Player'] = df_prediction2['Player'].astype('category')
df_prediction2['MP'] = df_prediction2.MP.fillna(df_prediction2.MP_preds)
feature_cols = [col for col in df_prediction2.columns if col not in ['Season', 'Date', 'MP_preds', 'Res_PTS']]
df_prediction2 = df_prediction2[df_prediction2.Date == now][feature_cols]
df_prediction2["Res_PTS_proj"] = res_model_RG.predict(df_prediction2)

df_prediction2['pred_prob'] = res_model_CLF.predict_proba(df_prediction2.drop('Res_PTS_proj', axis=1))[:,1]
df_prediction2['pred_class'] = (df_prediction2['pred_prob'] > 0.5).astype(int)

# Setup Today's Picks
df_lines = df_lines[df_lines.Date == now][['Team', 'Player', 'PTS_line']]
df_prediction = df_prediction.merge(df_lines, on=['Team', 'Player'])
df_prediction = df_prediction.merge(df_prediction2[['Team', 'Player', 'Res_PTS_proj', 'pred_prob', 'pred_class']], on=['Team', 'Player'])

tds_picks = df_prediction[~(df_prediction['PTS_line'].isnull())].rename(columns={"MP_proj": "MP"})\
            [['Team', 'Player', 'MP', 'PTS_line', 'PTS_proj', 'Res_PTS_proj', 'pred_prob', 'pred_class']]

tds_picks['O/U'] = '-'
tds_picks['O/U'] = np.where(((tds_picks.pred_class == 1) & (tds_picks.PTS_proj > tds_picks.PTS_line) & (tds_picks.Res_PTS_proj > 0)), 'O', tds_picks['O/U'])
tds_picks['O/U'] = np.where(((tds_picks.pred_class == 0) & (tds_picks.PTS_proj < tds_picks.PTS_line) & (tds_picks.Res_PTS_proj < 0)), 'U', tds_picks['O/U'])

tds_picks = tds_picks.sort_values(['O/U', 'Team', 'Player'], ascending=[False, False, False])
if tds_picks.shape[0] >= 50:
    print(tds_picks.shape[0], 'rows')
    for tm in tds_picks.Team.unique():
        display(tds_picks[tds_picks.Team == tm])
else:
    display(tds_picks)
tds_picks.insert(0, 'Date', pd.to_datetime(now))
partition_save_df(tds_picks, f"../tables/{YEAR}/gmday_preds_PTS.csv")

113 rows


,Team,Player,MP,PTS_line,PTS_proj,Res_PTS_proj,pred_prob,pred_class,O/U
107,SAC,Zach LaVine,26.354628,17.5,16.032999,-0.555905,0.314506,0,U
103,SAC,Malik Monk,24.576942,13.5,14.251740,1.945208,0.611189,1,O
100,SAC,DeMar DeRozan,34.751778,19.5,20.458120,1.837899,0.681094,1,O
106,SAC,Russell Westbrook,27.554213,14.5,13.957262,3.043906,0.628033,1,-
105,SAC,Precious Achiuwa,26.141809,7.5,7.382320,4.021583,0.777391,1,-
104,SAC,Maxime Raynaud,19.679895,6.5,6.764137,-1.085753,0.399542,0,-
102,SAC,Domantas Sabonis,32.572163,17.5,15.278681,2.442941,0.561787,1,-
101,SAC,Dennis Schroder,30.914150,14.5,12.227535,3.923611,0.641725,1,-


,Team,Player,MP,PTS_line,PTS_proj,Res_PTS_proj,pred_prob,pred_class,O/U
96,PHO,Jordan Goodwin,20.793865,8.5,7.606244,-1.576536,0.463316,0,U
95,PHO,Jalen Green,14.948758,12.5,7.817568,-5.970525,0.059788,0,U
97,PHO,Mark Williams,26.265177,10.5,11.701834,1.414486,0.559473,1,O
99,PHO,Royce O'Neale,30.203722,9.5,9.885855,-0.802900,0.622808,1,-
98,PHO,Oso Ighodaro,20.496599,5.5,5.545911,-0.027774,0.469284,0,-
94,PHO,Grayson Allen,32.114532,18.5,17.055044,0.752710,0.453450,0,-
93,PHO,Dillon Brooks,32.828773,21.5,20.015123,0.569532,0.555975,1,-
92,PHO,Collin Gillespie,29.905657,14.5,14.286736,-1.137729,0.523115,1,-


,Team,Player,MP,PTS_line,PTS_proj,Res_PTS_proj,pred_prob,pred_class,O/U
85,PHI,Jared McCain,18.460241,6.5,6.271284,-0.388089,0.474493,0,U
91,PHI,VJ Edgecombe,34.848240,12.5,14.196393,1.135515,0.621773,1,O
90,PHI,Tyrese Maxey,37.365299,26.5,28.686293,1.902614,0.660417,1,O
88,PHI,Paul George,32.577618,14.5,16.807789,1.932690,0.563207,1,O
87,PHI,Kelly Oubre Jr.,34.032059,11.5,14.630239,3.167320,0.652991,1,O
86,PHI,Joel Embiid,33.542572,26.5,29.018179,1.740541,0.685182,1,O
89,PHI,Quentin Grimes,24.791283,7.5,9.177062,-1.166725,0.471655,0,-


,Team,Player,MP,PTS_line,PTS_proj,Res_PTS_proj,pred_prob,pred_class,O/U
81,OKC,Isaiah Hartenstein,18.438568,7.5,6.542544,-0.438915,0.369302,0,U
79,OKC,Cason Wallace,22.560719,7.5,5.607897,-1.593956,0.260017,0,U
78,OKC,Aaron Wiggins,28.516251,12.5,12.252169,-0.696501,0.427931,0,U
84,OKC,Shai Gilgeous-Alexander,36.466560,32.5,33.004688,2.861444,0.609750,1,O
83,OKC,Luguentz Dort,32.352879,9.5,9.863848,2.167576,0.628118,1,O
82,OKC,Isaiah Joe,22.479677,9.5,9.894602,-1.546998,0.331433,0,-
80,OKC,Chet Holmgren,31.750095,18.5,18.239647,0.554847,0.587964,1,-


...

,Team,Player,MP,PTS_line,PTS_proj,Res_PTS_proj,pred_prob,pred_class,O/U
76,MIN,Naz Reid,25.600803,12.5,13.116174,1.635525,0.611998,1,O
75,MIN,Julius Randle,33.175251,18.5,19.763111,1.028651,0.565649,1,O
74,MIN,Jaden McDaniels,31.921116,12.5,13.107297,1.703484,0.598379,1,O
72,MIN,Anthony Edwards,36.975342,25.5,27.256495,1.774201,0.690462,1,O
77,MIN,Rudy Gobert,30.612780,8.5,9.496190,0.081146,0.409686,0,-
73,MIN,Donte DiVincenzo,31.567741,12.5,13.084131,-0.963921,0.516160,1,-


,Team,Player,MP,PTS_line,PTS_proj,Res_PTS_proj,pred_prob,pred_class,O/U
70,MIL,Myles Turner,30.513281,14.5,16.142298,0.028137,0.584655,1,O
67,MIL,Cole Anthony,19.392696,8.5,8.766709,0.517172,0.512985,1,O
71,MIL,Ryan Rollins,32.961552,22.5,17.984272,0.344342,0.505571,1,-
69,MIL,Kyle Kuzma,30.533031,15.5,14.508421,0.485269,0.344628,0,-
68,MIL,Gary Trent Jr.,19.381313,8.5,7.105974,0.244376,0.525954,1,-
66,MIL,Bobby Portis,31.050661,18.5,17.142426,4.665755,0.717984,1,-


,Team,Player,MP,PTS_line,PTS_proj,Res_PTS_proj,pred_prob,pred_class,O/U
61,MIA,Kasparas Jakucionis,24.119625,5.5,7.037666,0.452897,0.514022,1,O
59,MIA,Bam Adebayo,33.946301,19.5,20.413481,2.487547,0.594217,1,O
58,MIA,Andrew Wiggins,31.528830,15.5,17.039875,1.692022,0.505320,1,O
65,MIA,Simone Fontecchio,20.278744,8.5,10.328012,0.395774,0.461387,0,-
64,MIA,Pelle Larsson,28.739521,11.5,12.340618,0.904849,0.414399,0,-
63,MIA,Norman Powell,31.162100,21.5,23.094824,0.551130,0.266412,0,-
62,MIA,Nikola Jovic,22.254356,8.5,10.674971,1.109777,0.493635,0,-
60,MIA,Jaime Jaquez Jr.,29.310381,14.5,14.712004,1.380330,0.485551,0,-


,Team,Player,MP,PTS_line,PTS_proj,Res_PTS_proj,pred_prob,pred_class,O/U
57,HOU,Reed Sheppard,23.598673,12.5,12.739207,1.048506,0.540796,1,O
56,HOU,Kevin Durant,38.613670,25.5,30.730976,2.856693,0.563838,1,O
55,HOU,Josh Okogie,27.077105,6.5,6.779051,1.494623,0.519299,1,O
54,HOU,Jabari Smith Jr.,36.086617,14.5,14.757082,1.458726,0.657950,1,O
53,HOU,Amen Thompson,37.790493,16.5,18.490061,1.400625,0.654500,1,O
52,HOU,Alperen Sengun,34.890526,20.5,22.396181,-0.267269,0.583675,1,-


../tables/2025/gmday_preds_PTS.csv saved!


In [144]:
# tds_picks['PTS_proj_mag'] = tds_picks.PTS_proj - tds_picks.PTS_line
# print('O:', tds_picks[tds_picks.PTS_proj_mag > 0].shape[0] / tds_picks.shape[0])
# print('U:',tds_picks[tds_picks.PTS_proj_mag < 0].shape[0] / tds_picks.shape[0])

# plt.figure(figsize=(10,6))
# hist_col = 'PTS_proj_mag'
# plt.hist(tds_picks[hist_col], bins=30, color='skyblue', edgecolor='black')
# plt.title(f'Histogram of {hist_col}')
# plt.xlabel(hist_col)
# plt.ylabel('Frequency')
# plt.grid(axis='y', alpha=0.75)
# plt.show()
# tds_picks = tds_picks.drop('PTS_proj_mag', axis=1)

In [32]:
# print('O:',tds_picks[tds_picks.Res_PTS_proj > 0].shape[0] / tds_picks.shape[0])
# print('U:',tds_picks[tds_picks.Res_PTS_proj < 0].shape[0] / tds_picks.shape[0])

# plt.figure(figsize=(10,6))
# hist_col = 'Res_PTS_proj'
# plt.hist(tds_picks[hist_col], bins=30, color='skyblue', edgecolor='black')
# plt.title(f'Histogram of {hist_col}')
# plt.xlabel(hist_col)
# plt.ylabel('Frequency')
# plt.grid(axis='y', alpha=0.75)
# plt.show()

In [33]:
# print('O:', tds_picks[tds_picks.pred_class == 1].shape[0] / tds_picks.shape[0])
# print('U:', tds_picks[tds_picks.pred_class == 0].shape[0] / tds_picks.shape[0])

# plt.figure(figsize=(10,6))
# hist_col = 'pred_prob'
# plt.hist(tds_picks[hist_col], bins=30, color='skyblue', edgecolor='black')
# plt.title(f'Histogram of {hist_col}')
# plt.xlabel(hist_col)
# plt.ylabel('Frequency')
# plt.grid(axis='y', alpha=0.75)
# plt.show()

# Misc.

In [34]:
# # Historical Percentages
# df_yesterday = pd.read_csv(f'../tables/{YEAR}/gmday_preds_{tgt_stat}.csv')
# df_yesterday['Date'] = pd.to_datetime(df_yesterday.Date)
# df_yesterday = df_yesterday.rename(columns={"MP": "MP_proj"})

# df_gms = pd.read_csv(f"../tables/{YEAR}/season_gamelogs.csv")
# df_gms['Date'] = pd.to_datetime(df_gms.Date)

# df_yesterday = df_yesterday.merge(df_gms[['Date', 'Team', 'Player', tgt_stat, 'MP']], on=['Date', 'Team', 'Player'])
# df_yesterday = df_yesterday[['Date', 'Team', 'Player', 'MP', 'MP_proj', f'{tgt_stat}_line', f'{tgt_stat}_proj', tgt_stat]][df_yesterday.MP > 0]

# df_yesterday['Diff'] = df_yesterday[f'{tgt_stat}_proj'] - df_yesterday[f'{tgt_stat}_line']
# df_yesterday['Diff2'] = abs(df_yesterday[f'{tgt_stat}_proj'] - df_yesterday[tgt_stat])
# df_yesterday['Act_Res'] = np.where(df_yesterday[tgt_stat] > df_yesterday[f'{tgt_stat}_line'], 'O', 'U')
# df_yesterday['Pred_Res'] = np.where(df_yesterday[f'{tgt_stat}_proj'] > df_yesterday[f'{tgt_stat}_line'], 'O', 'U')
# df_yesterday['ParlayHit'] = np.where(df_yesterday['Act_Res'] == df_yesterday['Pred_Res'], 1, 0)
# df_yesterday['Diff3'] = abs(df_yesterday['MP_proj'] - df_yesterday['MP'])
# df_yesterday['InRMSE_Range'] = np.where(df_yesterday['Diff3'] <= 5, 1, 0)

# for day in df_gms.Date.unique():
#     df_temp = df_yesterday[df_yesterday.Date == day]
#     if df_temp.shape[0] > 0:
#         print(f"{day.date()} Total PTS Accuracy:", f"{(df_temp.ParlayHit == 1).sum()}/{df_temp.shape[0]}", ((df_temp.ParlayHit == 1).sum() / df_temp.shape[0]))
#         print(f"{day.date()} Total MP Accuracy:", f"{(df_temp.InRMSE_Range == 1).sum()}/{df_temp.shape[0]}", ((df_temp.InRMSE_Range == 1).sum() / df_temp.shape[0]), "\n")

In [35]:
# df_mins = df.copy()
# df_mins = setup_df_mins(df_mins)

# train_summary = df_mins.drop(['Season', 'Date', 'MP'], axis=1).describe().T
# gameday_summary = df_prediction_mins[df_prediction_mins.Date == now].drop(['Season', 'Date', 'MP'], axis=1).describe().T
# display(train_summary[['mean','std']])
# display(gameday_summary[['mean','std']])

In [36]:
# df_main = df.copy()
# df_main = setup_df_main(df_main, tgt_stat)

# train_summary = df_main.drop(['Season', 'Team', 'Opp', 'Player', 'Pos', 'Date', 'PTS'], axis=1).describe().T
# gameday_summary = df_prediction[(~df_prediction.PTS_line.isnull())].drop(['Pos', 'PTS_proj', 'Res_PTS_proj', 'pred_prob', 'pred_class', 'PTS_line'], axis=1).describe().T
# display(train_summary[['mean','std']])
# display(gameday_summary[['mean','std']])

In [37]:
# df_res = df[(~df.PTS_line.isnull())].copy()
# df_res = setup_df_res(df_res)

# train_summary = df_res.drop(['Date', 'Res_PTS'], axis=1).describe().T
# gameday_summary = df_prediction2[(~df_prediction2.PTS_line.isnull())].describe().T
# display(train_summary[['mean','std']])
# display(gameday_summary[['mean','std']])

In [38]:
# for col in mins_train_df.drop(['Season', 'Date', 'MP'], axis=1).columns:
#     if col not in ['Team', 'Player', 'Opp', 'Pos']:
#         PartialDependenceDisplay.from_estimator(
#             mins_model,
#             mins_train_df.drop(['Season', 'Date', 'MP'], axis=1),
#             features=[col],
#             grid_resolution=25
#         )
        
# ##################
# features = [('role')]
# PartialDependenceDisplay.from_estimator(
#     mins_model,
#     mins_train_df.drop(['Season', 'Date', 'MP'], axis=1),
#     features=features,
#     grid_resolution=25
# )